In [1]:
import sys
from pathlib import Path

# Get the absolute path of the current notebook
notebook_path = Path().resolve()

# Get the project root directory (which is the parent of the 'notebooks' directory)
project_root = notebook_path.parent

# Add BOTH the project root and the src directory to the Python path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
if str(project_root / 'src') not in sys.path:
    sys.path.append(str(project_root / 'src'))

# Now, we can import our modules
from src.data_handling import DataHandler
from src.mlp_model import MLPModel

Using MPS device (Apple Silicon GPU)


#### Initialize DataHandler
This cell creates an instance of the `DataHandler` class. It will load the dataset, define the feature/target sets, and pre-fit the necessary data processing objects. The test year is set to 2020.

In [2]:
dh = DataHandler(test_year=2020)

DataHandler initialized - Using 52 features - Test year: 2020


### **Model: MLP with 2 Hidden Layers**

#### Initialize MLPModel Handler
Here, we create an instance of the `MLPModel` class, explicitly setting `depth=2`. The `model_name` ensures all artifacts for this architecture are saved to unique files.

In [3]:
mlp_depth2 = MLPModel(dh, model_name='mlp_depth2', depth=2)

MLPModel initialized with model name mlp_depth2 and depth 2.
Optuna study will be stored in  : /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/optuna/mlp_depth2_study.pkl
Trained model will be stored in : /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/models/mlp_depth2_model.pth
Final preds will be stored in   : /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/preds/mlp_depth2_preds.csv


#### Run Optuna Hyperparameter Study (Depth=2)
This cell runs the Optuna study. For this specific 2-layer architecture, it will find the optimal learning rate, weight decay, batch size, and the number of neurons and dropout rates for both hidden layers.

In [4]:
mlp_depth2.run_optuna_study(
    n_trials=50,
    timeout=30,               # Stop study after 30 minutes
    max_epochs=40,            # Max epochs to train each trial
    min_resource=4,          # Min epochs before pruning
    reduction_factor=2,
    use_pca=False
)

[I 2025-10-23 10:55:34,169] A new study created in memory with name: mlp_depth2_study
[I 2025-10-23 10:55:53,523] Trial 8 pruned. 
[I 2025-10-23 10:55:53,731] Trial 5 pruned. 
[I 2025-10-23 10:55:54,313] Trial 2 pruned. 
[I 2025-10-23 10:56:03,228] Trial 4 pruned. 
[I 2025-10-23 10:56:07,028] Trial 6 pruned. 
[I 2025-10-23 10:56:07,715] Trial 1 pruned. 
[I 2025-10-23 10:56:09,927] Trial 11 pruned. 
[I 2025-10-23 10:56:17,241] Trial 12 pruned. 
[I 2025-10-23 10:56:23,996] Trial 14 pruned. 
[I 2025-10-23 10:56:24,310] Trial 15 pruned. 
[I 2025-10-23 10:56:32,434] Trial 19 pruned. 
[I 2025-10-23 10:56:33,586] Trial 17 pruned. 
[I 2025-10-23 10:56:41,294] Trial 18 pruned. 
[I 2025-10-23 10:56:43,293] Trial 16 pruned. 
[I 2025-10-23 10:56:52,979] Trial 10 pruned. 
[I 2025-10-23 10:56:55,348] Trial 7 finished with value: 1.0158263511127896 and parameters: {'learning_rate': 0.07773599703457267, 'weight_decay': 1.4686794058846736e-05, 'batch_size': 256, 'n_hidden_1': 104, 'dropout_rate_1': 0.1

--------------------
Study concluded and saved to /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/optuna/mlp_depth2_study.pkl.
Best trial: 31
Best loss: 1.0008552471796672
Best params: {'learning_rate': 0.013638764634900312, 'weight_decay': 7.283795869669707e-06, 'batch_size': 256, 'n_hidden_1': 104, 'dropout_rate_1': 0.2999430481001182, 'n_hidden_2': 64, 'dropout_rate_2': 0.08380730710710263}
Best epoch: 36


#### Train Final Model (Depth=2)
Using the best hyperparameters and the optimal number of training epochs found by Optuna, this cell trains the final 2-layer MLP on the entire training dataset. The model's weights (`.pth` file) are saved to the results directory.

In [5]:
mlp_depth2.train_final_model(patience=10) # Stop if loss doesn't improve for 10 epochs

Training mlp_depth2 with depth 2 for 36 epochs with patience of 10 epochs.
Using batch size: 256 and best params: {'learning_rate': 0.013638764634900312, 'weight_decay': 7.283795869669707e-06, 'batch_size': 256, 'n_hidden_1': 104, 'dropout_rate_1': 0.2999430481001182, 'n_hidden_2': 64, 'dropout_rate_2': 0.08380730710710263}.
Epoch    0, Loss: 1.044705
Epoch    1, Loss: 1.005406
Epoch    2, Loss: 0.997768
Epoch    3, Loss: 0.997712
Epoch    4, Loss: 0.997188
Epoch    5, Loss: 0.996592
Epoch    6, Loss: 0.998034
Epoch    7, Loss: 0.995895
Epoch    8, Loss: 0.993838
Epoch    9, Loss: 0.992954
Epoch   10, Loss: 0.994985
Epoch   11, Loss: 0.993056
Epoch   12, Loss: 0.994317
Epoch   13, Loss: 0.994425
Epoch   14, Loss: 0.993872
Epoch   15, Loss: 0.993259
Epoch   16, Loss: 0.995204
Epoch   17, Loss: 0.994015
Epoch   18, Loss: 0.993140
Epoch   19, Loss: 0.993627
Early stopping at epoch 19.
Training completed in 3.29 seconds.
Model saved to /Users/arvindsuresh/Documents/Github/Election-predicti

#### Generate Final Predictions (Depth=2)
Finally, we load the trained 2-layer model and use it to make predictions on the held-out 2020 test set.

In [6]:
# Make predictions on the 2020 test data
preds_depth2 = mlp_depth2.make_final_predictions()
print("\nMLP (Depth=2) Predictions (first 5 rows):")
print(preds_depth2[:5])

mlp_depth2 predictions saved to: /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/preds/mlp_depth2_preds.csv.

MLP (Depth=2) Predictions (first 5 rows):
[[0.15294294 0.2773426  0.02739584 0.54231864]
 [0.14686209 0.29007718 0.04715114 0.51590955]
 [0.19786239 0.19287218 0.01319368 0.5960718 ]
 [0.11619189 0.23342094 0.01778831 0.6325988 ]
 [0.12058429 0.2607012  0.03078086 0.5879336 ]]
